In [3]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [9]:
import os,sys
data_path = '../../docs/data/actor.json'

In [17]:
import pymongo
import json

## 1. create connection
conn = pymongo.MongoClient(host = 'localhost', port = 27017)
# print(conn)

## 2. create database 
movie_db = conn.movie_db
# print(movie_db)

## 3. create collection
actors = movie_db.actor_col
# print(actors)
# print(actors.name)

## 4. json file open and write
with open(data_path, 'r', encoding='utf8') as f:
    json_data = json.load(f)
    actors.insert_many(json_data)

## print documents count 
print(actors.estimated_document_count())

1903


In [24]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)
query = {
    '소속사' : '호두엔터테인먼트'
}
for doc in actors.find(query):
    pp.pprint(doc)

{   '_id': ObjectId('5f28b4e9b57df343aea25013'),
    '다른 이름': '전이다',
    '랭킹': 4,
    '생년월일': '1976-08-10',
    '성별': '여',
    '소속사': '호두엔터테인먼트',
    '신장/체중': '171cm, 49kg',
    '이름': '전혜진',
    '직업': '배우',
    '출연영화': ['백두산', '비스트', '뺑반', '불한당: 나쁜 놈들의 세상', '택시운전사'],
    '취미': '영화감상, 노래 부르기',
    '특기': '수영',
    '학교': '상명대학교 영화과',
    '흥행지수': 17976}
{   '_id': ObjectId('5f28b4e9b57df343aea2506c'),
    '랭킹': 93,
    '생년월일': '1967-01-17',
    '성별': '남',
    '소속사': '호두엔터테인먼트',
    '신장/체중': '180cm, 80kg',
    '이름': '송강호',
    '직업': '배우',
    '출연영화': ['나랏말싸미', '기생충', '설국열차', '밀양', '의형제', '쉬리'],
    '학교': '부산경상대학 방송연예',
    '흥행지수': 3162}
{   '_id': ObjectId('5f28b4e9b57df343aea250a7'),
    '랭킹': 152,
    '생년월일': '1975-03-02',
    '성별': '남',
    '소속사': '호두엔터테인먼트',
    '신장/체중': '180cm, 70kg',
    '이름': '이선균',
    '직업': '배우',
    '출연영화': ['기생충', '미스터 주: 사라진 VIP', '끝까지 간다'],
    '취미': '만화책보기',
    '학교': '한국예술종합학교 연극',
    '흥행지수': 1543}


### 1. Sort

In [43]:
docs = actors.find({}).sort('이름')
docs = actors.find({}).sort('이름', pymongo.DESCENDING)

## sort by multiple field
docs = actors.find({}).sort( [ ('이름', pymongo.ASCENDING) , ('랭킹', pymongo.DESCENDING) ] )
for idx, actor in enumerate(docs,1):
    # pp.pprint(actor)
    # print(idx, actor['이름'], actor['랭킹'])
    pass

## 2. Exists 
* is exist value of specific field

In [46]:
query = actors.find({
    '특기' :{
        '$exists' : True
    }
}).count()
query = actors.find({
    '특기' :{
        '$exists' : False
    }
}).count()
pp.pprint(query)

1725


In [64]:
query = actors.find({
    '생년월일' :{
        '$exists' : True
    }
}).count()
# query = actors.find({},{'이름' : 1, '생년월일' : 1, '_id' : 0})
# for doc in query:
#     print(doc['이름'], doc['생년월일'])
print(query)

1268


In [73]:
## find whose '다른 이름' is '조원준'
query = actors.find(
    {
        '다른 이름' : {
            '$exists' : True
        },
        '다른 이름' : '조원준'
    },
    {
        '다른 이름' : 1,
        '랭킹' : 1,
        '이름' : 1,
        '_id' : 0,
    }
)
for doc in query:
    pp.pprint(doc)

{'다른 이름': '조원준', '랭킹': 9, '이름': '조진웅'}


## 3. search by field range
* \$gt, \$gte, \$lt, \$lte

In [81]:
## find 10000 <= '흥행지수'  <= 30000
query = actors.find(
    {
        '흥행지수' : {
            '$gte' : 10000,
            '$lte' : 30000,
        }
    },
    {
        '이름' : 1,
        '흥행지수' : 1,
        '_id' : 0,
    }
)
print(query.count())
for actor in query:
    pp.pprint(actor)

19
{'이름': '마동석', '흥행지수': 19019}
{'이름': '전혜진', '흥행지수': 17976}
{'이름': '이성민', '흥행지수': 15213}
{'이름': '김아중', '흥행지수': 15208}
{'이름': '박정민', '흥행지수': 14534}
{'이름': '강기영', '흥행지수': 13974}
{'이름': '조진웅', '흥행지수': 13472}
{'이름': '조정석', '흥행지수': 13192}
{'이름': '배수지', '흥행지수': 12646}
{'이름': '유아인', '흥행지수': 12422}
{'이름': '임윤아', '흥행지수': 12343}
{'이름': '고두심', '흥행지수': 11496}
{'이름': '장기용', '흥행지수': 11406}
{'이름': '권상우', '흥행지수': 11263}
{'이름': '곽도원', '흥행지수': 11254}
{'이름': '유해진', '흥행지수': 10910}
{'이름': '최유화', '흥행지수': 10898}
{'이름': '박인환', '흥행지수': 10669}
{'이름': '정해인', '흥행지수': 10447}


## 4. nor and or

In [98]:
## or
## 흥행지수 >= 15000 or 랭킹 <= 20
query = actors.find(
    {
        '$or': 
            [{'흥행지수' : {
                '$gte' : 15000
            }},
            {'랭킹' : {
                '$lte' : 20
            }}]
    },
    {
        '_id' : 0,
        '랭킹' : 1,
        '흥행지수' : 1,
        '이름' : 1
    }
)

for actor in query:
    pp.pprint(actor)

{'랭킹': 1, '이름': '이병헌', '흥행지수': 45760}
{'랭킹': 2, '이름': '하정우', '흥행지수': 31124}
{'랭킹': 3, '이름': '마동석', '흥행지수': 19019}
{'랭킹': 4, '이름': '전혜진', '흥행지수': 17976}
{'랭킹': 5, '이름': '이성민', '흥행지수': 15213}
{'랭킹': 6, '이름': '김아중', '흥행지수': 15208}
{'랭킹': 7, '이름': '박정민', '흥행지수': 14534}
{'랭킹': 8, '이름': '강기영', '흥행지수': 13974}
{'랭킹': 9, '이름': '조진웅', '흥행지수': 13472}
{'랭킹': 10, '이름': '조정석', '흥행지수': 13192}
{'랭킹': 11, '이름': '배수지', '흥행지수': 12646}
{'랭킹': 12, '이름': '유아인', '흥행지수': 12422}
{'랭킹': 13, '이름': '임윤아', '흥행지수': 12343}
{'랭킹': 14, '이름': '고두심', '흥행지수': 11496}
{'랭킹': 15, '이름': '장기용', '흥행지수': 11406}
{'랭킹': 16, '이름': '권상우', '흥행지수': 11263}
{'랭킹': 17, '이름': '곽도원', '흥행지수': 11254}
{'랭킹': 18, '이름': '유해진', '흥행지수': 10910}
{'랭킹': 19, '이름': '최유화', '흥행지수': 10898}
{'랭킹': 20, '이름': '박인환', '흥행지수': 10669}


In [119]:
## nor 
## 흥행지수 >= 15000, 성별 = 남
query = actors.find(
    {
        '$nor' : [
            {
                '흥행지수' :{
                    '$lte' : 15000
                }
            },
            {
                '성별' : {
                    '$ne' : '남'
                }
            }
        ]
    },
    {
        '_id' : 0,
        '이름' : 1,
        '성별' : 1,
        '흥행지수' : 1
    }
)
for actor in query:
    pp.pprint(actor)

{'성별': '남', '이름': '이병헌', '흥행지수': 45760}
{'성별': '남', '이름': '하정우', '흥행지수': 31124}
{'성별': '남', '이름': '마동석', '흥행지수': 19019}
{'성별': '남', '이름': '이성민', '흥행지수': 15213}


## 5. aggregate and sortByCount

In [132]:
query = actors.aggregate([
    {
        '$sortByCount' : '$소속사',
    },
    {
        '$match' : {
            'count': {
                '$gte' : 4
            }
        }
    }
])
for actor in query:
    pp.pprint(actor)

{'_id': None, 'count': 1723}
{'_id': '아티스트컴퍼니', 'count': 16}
{'_id': '싸이더스 HQ', 'count': 11}
{'_id': '나무액터스', 'count': 10}
{'_id': '열음엔터테인먼트', 'count': 8}
{'_id': 'SM엔터테인먼트', 'count': 5}
{'_id': '키이스트', 'count': 5}
{'_id': '큐브엔터테인먼트', 'count': 5}
{'_id': '예당엔터테인먼트', 'count': 4}
{'_id': 'SM C&amp;C', 'count': 4}
{'_id': '노아엔터테인먼트', 'count': 4}
{'_id': '(주)YG엔터테인먼트', 'count': 4}
{'_id': '㈜사람엔터테인먼트', 'count': 4}
{'_id': 'JYP엔터테인먼트', 'count': 4}
{'_id': '지티비엔터테인먼트', 'count': 4}


## 6. search inside of list, which is value of the key 
* order and value must correct
* if don't care about order, use \$all 

In [156]:
## Find including '반도' in '출연영화'
match = {'출연영화' : '반도'}

## Find including '1987'or '반도' in '출연영화'
match = { '$or' : 
    [{'출연영화' : '반도'}, {'출연영화' : '1987'}]
}
## Find including '남산의 부장들' and '공작' in '출연영화'

match = {                                       ## no result! because of order 
    '출연영화' : ['남산의 부장들','공작']
    }    

match = { '출연영화' :{                           ## this can find what i wanted
     '$all' : ['남산의 부장들','공작']
}}

## Find first attribute of '출연영화' is '결백'
match = {
    '출연영화.0' : '결백'
}

## Find second attribute of '출연영화' is '밀양'
match = {
    '출연영화.1' : '밀양'
}

## Find who have 6 attributes in '출연영화'
match = {
    '출연영화' : {
        '$size' : 4
    }
}

field = {'이름' : 1, '출연영화' : 1, '_id' : 0}
for actor in actors.find(match, field):
    pp.pprint(actor)

{'이름': '권상우', '출연영화': ['히트맨', '신의 한 수: 귀수편', '두번할까요', '탐정: 리턴즈']}
{'이름': '최유화', '출연영화': ['봉오동 전투', '타짜: 원 아이드 잭', '비밀은 없다', '최악의 하루']}
{'이름': '강동원', '출연영화': ['반도', '의형제', '인랑', '1987']}
{'이름': '이레', '출연영화': ['반도', '너의 이름은.', '증인', '개를 훔치는 완벽한 방법']}
{'이름': '성유빈', '출연영화': ['봉오동 전투', '윤희에게', '생일', '아이 캔 스피크']}
{'이름': '윤경호', '출연영화': ['시동', '정직한 후보', '배심원들', '내안의 그놈']}
{'이름': '김남길', '출연영화': ['클로젯', '무뢰한', '기묘한 가족', '판도라']}
{'이름': '류승범', '출연영화': ['타짜: 원 아이드 잭', '죽거나 혹은 나쁘거나', '부당거래', '신세계']}
{'이름': '강신일', '출연영화': ['저 산 너머', '블랙머니', '집 이야기', '판도라']}
{'이름': '정웅인', '출연영화': ['가장 보통의 연애', '슈팅걸스', '베테랑', '시간이탈자']}
{'이름': '구교환', '출연영화': ['메기', '반도', '꿈의 제인', '우리 손자 베스트']}
{'이름': '장영남', '출연영화': ['변신', '국제시장', '증인', '미쓰백']}
{'이름': '나문희', '출연영화': ['감쪽같은 그녀', '정직한 후보', '아이 캔 스피크', '수상한 그녀']}
{'이름': '신구', '출연영화': ['천문: 하늘에 묻는다', '8월의 크리스마스', '비밥바룰라', '해빙']}
{'이름': '이채은', '출연영화': ['가장 보통의 연애', '비밥바룰라', '부당거래', '어쩌다, 결혼']}
{'이름': '이광수', '출연영화': ['나의 특별한 형제', '타짜: 원 아이드 잭', '마이 리틀 히어로', '탐정: 리턴즈']}
{'이름': 

## 7. limit

In [171]:
# '직업' is '가수' and '흥행지수' top 10
field = {'이름' : 1, '직업' : 1, '흥행지수' : 1,  '_id' : 0}
query = actors.find(
    {
        '직업' : '가수'
    },
    field
).sort('흥행지수', pymongo.DESCENDING).limit(10)
for actor in query:
    pp.pprint(actor)

{'이름': '배수지', '직업': '가수', '흥행지수': 12646}
{'이름': '임윤아', '직업': '가수', '흥행지수': 12343}
{'이름': '최민호', '직업': '가수', '흥행지수': 2008}
{'이름': 'RM', '직업': '가수', '흥행지수': 1061}
{'이름': '진', '직업': '가수', '흥행지수': 909}
{'이름': '슈가', '직업': '가수', '흥행지수': 758}
{'이름': '준호', '직업': '가수', '흥행지수': 518}
{'이름': '지민', '직업': '가수', '흥행지수': 455}
{'이름': '김소혜', '직업': '가수', '흥행지수': 324}
{'이름': '뷔', '직업': '가수', '흥행지수': 303}


In [175]:
# who has 국가부도의 날 in 출연영화, top 10 흥행지수
field = {'이름' : 1, '출연영화' : 1, '흥행지수' : 1,  '_id' : 0}
query = actors.find(
    {
        '출연영화' : '국가부도의 날'
    },
    field
).sort('흥행지수', pymongo.DESCENDING).limit(10)
for actor in query:
    pp.pprint(actor)

{   '이름': '유아인',
    '출연영화': ['＃살아있다', '베테랑', '버닝', '국가부도의 날', '우아한 거짓말'],
    '흥행지수': 12422}
{   '이름': '조우진',
    '출연영화': ['봉오동 전투', '내부자들', '돈', '국가부도의 날', '어쩌다, 결혼', '창궐'],
    '흥행지수': 8725}
{   '이름': '허준호',
    '출연영화': ['천문: 하늘에 묻는다', '퍼펙트맨', '결백', '국가부도의 날', '인랑'],
    '흥행지수': 6739}
{   '이름': '전배수',
    '출연영화': ['＃살아있다', '광해, 왕이 된 남자', '기묘한 가족', '국가부도의 날'],
    '흥행지수': 1761}
{   '이름': '송영창',
    '출연영화': ['정직한 후보', '변호인', '베테랑', '말모이', '국가부도의 날', '해빙'],
    '흥행지수': 1715}
{'이름': '김민상', '출연영화': ['남산의 부장들', '국가부도의 날', '럭키'], '흥행지수': 121}
{   '이름': '염혜란',
    '출연영화': ['야구소녀', '걸캅스', '아이 캔 스피크', '증인', '미성년', '국가부도의 날'],
    '흥행지수': 90}
{'이름': '류덕환', '출연영화': ['난폭한 기록', '국가부도의 날', '천하장사 마돈나'], '흥행지수': 46}
{'이름': '김혜수', '출연영화': ['도둑들', '타짜', '국가부도의 날'], '흥행지수': 2}
{'이름': '박진주', '출연영화': ['스윙키즈', '국가부도의 날'], '흥행지수': 0}
